## Interactive Calculus Explorer: User-Defined Functions

This notebook allows you to visualize a **function**, its **derivative (tangent line)**, and its **definite integral (Riemann/Darboux sums)** by simply typing in a mathematical expression.

---

### How to Use:

1.  **Input your function:** Type your function (e.g., `x**2`, `np.sin(x)`, `np.exp(x)`) into the "Function f(x):" text box.
    * **Important:** Use `np.sin()`, `np.cos()`, `np.exp()`, `np.log()` for trigonometric, exponential, and logarithmic functions, as we're using NumPy.
2.  **Adjust the interval:** Use the "Start (a)" and "End (b)" sliders to define the x-interval.
3.  **Explore the derivative:**
    * Use the "Point for Derivative (x?)" slider to select a point on the curve.
    * Observe the **tangent line** at that point.
4.  **Explore the integral:**
    * Adjust the "Number of Rectangles (N)" slider to see how the approximation improves.
    * Choose between **Left, Right, Midpoint Riemann Sums**, or **Lower/Upper Darboux Sums** using the "Summation Method" dropdown.
    * See the approximate area displayed in the title.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

# Define the plotting function that will be controlled by the widgets
def interactive_calculus_plot(func_str, a, b, x0, N, method):
    # --- Function Definition and Error Handling ---
    try:
        # Safely evaluate the user's function string
        # Use a lambda function to define f(x) from the string
        # The 'x' variable will be an array when passed from np.linspace
        # We also make numpy functions available for eval
        f = lambda x: eval(func_str, {'np': np, 'x': x})

        # Test the function with a small array to catch immediate errors
        test_x = np.array([a, (a+b)/2, b])
        test_y = f(test_x)
        if np.any(np.isnan(test_y)) or np.any(np.isinf(test_y)):
            raise ValueError("Function output NaN or Inf for the given range.")

    except Exception as e:
        plt.figure(figsize=(10, 6))
        plt.text(0.5, 0.5, f"Error in function definition: {e}",
                 horizontalalignment='center', verticalalignment='center',
                 color='red', fontsize=14)
        plt.axis('off')
        plt.show()
        return

    # --- Plot Setup ---
    fig, ax = plt.subplots(figsize=(12, 7))

    # Generate x values for the main function plot over a slightly extended range
    x_plot = np.linspace(a - (b-a)*0.1, b + (b-a)*0.1, 400)
    y_plot = f(x_plot)
    ax.plot(x_plot, y_plot, label=f'$f(x) = {func_str}$', color='blue')

    # --- Derivative Visualization (Tangent Line) ---
    # Numeric approximation of the derivative (slope)
    # Using a small h for approximation: (f(x0 + h) - f(x0)) / h
    h_tangent = 1e-6 # A very small step for numerical derivative
    y0 = f(x0)

    # Handle potential errors if x0 is outside the domain of the function
    if np.isnan(y0) or np.isinf(y0):
        ax.text(0.5, 0.5, f"Cannot plot tangent: x?={x0:.2f} is outside function's domain.",
                 horizontalalignment='center', verticalalignment='center',
                 color='red', fontsize=12, transform=ax.transAxes)
    else:
        # Try to calculate derivative, add more robust error handling if needed
        try:
            slope_tangent = (f(x0 + h_tangent) - y0) / h_tangent
            tangent_x = np.linspace(x0 - (b-a)*0.1, x0 + (b-a)*0.1, 100) # Extend tangent line for visibility
            tangent_y = slope_tangent * (tangent_x - x0) + y0
            ax.plot(tangent_x, tangent_y, color='purple', linestyle='-', linewidth=2, label=f'Tangent at $x_0={x0:.2f}$ (slope: {slope_tangent:.2f})')
            ax.scatter([x0], [y0], color='green', zorder=5, s=100, label=f'Point $x_0=({x0:.2f},{y0:.2f})$')
        except Exception as e:
            ax.text(x0, y0, f"Derivative error at x?={x0:.2f}: {e}", color='orange', verticalalignment='bottom')

    # --- Integral Visualization (Riemann/Darboux Sums) ---
    dx = (b - a) / N

    # Calculate points for rectangle heights based on method
    if method == 'Left':
        x_rect_points = np.linspace(a, b - dx, N)
    elif method == 'Right':
        x_rect_points = np.linspace(a + dx, b, N)
    elif method == 'Midpoint':
        x_rect_points = np.linspace(a + dx/2, b - dx/2, N)
    elif method == 'Lower Darboux':
        # For general functions, this would require finding the actual min in each subinterval.
        # For monotonically increasing/decreasing, it simplifies to an endpoint.
        # Here, we approximate by choosing the smaller f(x) at interval endpoints.
        x_subintervals = np.linspace(a, b, N + 1)
        x_rect_points = []
        for i in range(N):
            sub_x = np.linspace(x_subintervals[i], x_subintervals[i+1], 50) # Sample points in subinterval
            min_idx = np.argmin(f(sub_x))
            x_rect_points.append(sub_x[min_idx])
        x_rect_points = np.array(x_rect_points)
    elif method == 'Upper Darboux':
        # Similar to Lower Darboux, find max in each subinterval.
        x_subintervals = np.linspace(a, b, N + 1)
        x_rect_points = []
        for i in range(N):
            sub_x = np.linspace(x_subintervals[i], x_subintervals[i+1], 50) # Sample points in subinterval
            max_idx = np.argmax(f(sub_x))
            x_rect_points.append(sub_x[max_idx])
        x_rect_points = np.array(x_rect_points)
    else:
        x_rect_points = np.linspace(a, b - dx, N) # Default to Left

    # Calculate heights and sum. Handle potential NaNs/Infs in heights.
    try:
        heights = f(x_rect_points)
        # Replace any non-finite values with 0 or a reasonable boundary to prevent plotting issues
        heights[np.isnan(heights)] = 0
        heights[np.isinf(heights)] = 0 # Or a very large/small number for visualization if desired

        riemann_sum = np.sum(heights * dx)
    except Exception as e:
        riemann_sum = float('nan') # Indicate calculation failed
        ax.text(0.5, 0.4, f"Integral calculation error: {e}",
                horizontalalignment='center', verticalalignment='center',
                color='red', fontsize=12, transform=ax.transAxes)

    # Draw the rectangles
    for i in range(N):
        # Adjust x-coordinate for rectangle plotting based on method
        if method == 'Left' or method == 'Lower Darboux':
            rect_x = x_rect_points[i]
        elif method == 'Right' or method == 'Upper Darboux':
            rect_x = x_rect_points[i] - dx
        elif method == 'Midpoint':
            rect_x = x_rect_points[i] - dx/2

        # Plot the rectangle only if height is finite and within reasonable bounds
        if np.isfinite(heights[i]) and abs(heights[i]) < 1e5: # Avoid drawing extremely tall rectangles
            ax.add_patch(plt.Rectangle((rect_x, 0), dx, heights[i],
                                       facecolor='lightgreen', edgecolor='green', alpha=0.7))

    # Shade the actual area (for visual reference only)
    # Using a high N for fill to make it smooth, regardless of current N
    x_fill_smooth = np.linspace(a, b, 200)
    y_fill_smooth = f(x_fill_smooth)
    # Filter out NaNs and Infs for filling
    valid_y_fill_smooth = np.isfinite(y_fill_smooth) & (np.abs(y_fill_smooth) < 1e5)
    ax.fill_between(x_fill_smooth[valid_y_fill_smooth], y_fill_smooth[valid_y_fill_smooth],
                    color='lightblue', alpha=0.2, label='Actual Area Region')

    # --- Plot Labels and Limits ---
    ax.set_title(f'Integral Approx: {riemann_sum:.4f} | Method: {method} | Rects: {N}')
    ax.set_xlabel('$x$')
    ax.set_ylabel('$f(x)$')
    ax.axhline(0, color='black', linewidth=0.5)
    ax.axvline(a, color='gray', linestyle='--', linewidth=0.8, label=f'Integral Start $a={a}$')
    ax.axvline(b, color='gray', linestyle='--', linewidth=0.8, label=f'Integral End $b={b}$')
    ax.grid(True, linestyle=':', alpha=0.7)
    ax.legend(loc='upper right')

    # Dynamic y-axis limits, but prevent extreme zooming for bad functions
    y_min_plot = np.min(y_plot[np.isfinite(y_plot)]) if np.any(np.isfinite(y_plot)) else -1
    y_max_plot = np.max(y_plot[np.isfinite(y_plot)]) if np.any(np.isfinite(y_plot)) else 1

    # Add some padding and clamp to reasonable values
    y_padding = (y_max_plot - y_min_plot) * 0.1
    ax.set_ylim(max(y_min_plot - y_padding, -100), min(y_max_plot + y_padding, 100))
    ax.set_xlim(x_plot.min(), x_plot.max())

    plt.show()

# --- Widgets Setup ---
func_input = widgets.Text(value='x**2 + 1',
                          placeholder='Type your function here (e.g., np.sin(x))',
                          description='Function f(x):',
                          layout=widgets.Layout(width='50%'))

a_slider = widgets.FloatSlider(min=-50, max=50, step=0.1, value=0.0, description='Start (a)',
                               continuous_update=True)

b_slider = widgets.FloatSlider(min=-50, max=50, step=0.1, value=2.0, description='End (b)',
                               continuous_update=True)

x0_slider = widgets.FloatSlider(min=-50, max=50, step=0.1, value=1.0, description='Point for Derivative (x?)',
                                continuous_update=True)

N_slider = widgets.IntSlider(min=1, max=1000, step=1, value=5, description='Number of Rectangles (N)',
                             continuous_update=True)

method_dropdown = widgets.Dropdown(options=['Left', 'Right', 'Midpoint', 'Lower Darboux', 'Upper Darboux'],
                                   value='Left',
                                   description='Summation Method:')

# Link b_slider min to a_slider value to ensure b > a
def update_b_min(*args):
    b_slider.min = a_slider.value + 0.1 if a_slider.value < b_slider.max else b_slider.value + 0.1
    if b_slider.value <= a_slider.value:
        b_slider.value = a_slider.value + 0.1

a_slider.observe(update_b_min, 'value')

# Link x0_slider limits to a and b sliders
def update_x0_limits(*args):
    x0_slider.min = a_slider.value
    x0_slider.max = b_slider.value
    if x0_slider.value < a_slider.value or x0_slider.value > b_slider.value:
        x0_slider.value = (a_slider.value + b_slider.value) / 2 # Reset to midpoint if outside bounds

a_slider.observe(update_x0_limits, 'value')
b_slider.observe(update_x0_limits, 'value')

# Display the interactive plot
interactive_plot = widgets.interactive(interactive_calculus_plot,
                                      func_str=func_input,
                                      a=a_slider,
                                      b=b_slider,
                                      x0=x0_slider,
                                      N=N_slider,
                                      method=method_dropdown)

display(interactive_plot)


interactive(children=(Text(value='x**2 + 1', description='Function f(x):', layout=Layout(width='50%'), placeho…